In [3]:
import pandas as pd
import numpy as np
import random

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

import rdkit
from rdkit.Chem import AllChem, rdFingerprintGenerator
from rdkit import Chem, DataStructs

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

import matplotlib.pyplot as plt

import xgboost as xgb

In [4]:
RANDOM_SEED = 42

In [7]:
def feature_engineer():
  train_df = pd.read_csv("./data/train.csv")
  test_df = pd.read_csv("./data/test.csv")
  train_df["mol"] = train_df["SMILES"].apply(lambda x: Chem.MolFromSmiles(x))
  test_df["mol"] = test_df["SMILES"].apply(lambda x: Chem.MolFromSmiles(x))
  
  train_df.drop(columns=["id", "SMILES"], inplace=True)
  test_df.drop(columns=["id", "SMILES"], inplace=True)
  
  
  fmgen = rdFingerprintGenerator.GetMorganGenerator()
  train_fps = train_df["mol"].apply(lambda x: fmgen.GetFingerprintAsNumPy(x))
  train_fps = np.stack(train_fps)
  test_fps = test_df["mol"].apply(lambda x: fmgen.GetFingerprintAsNumPy(x))
  test_fps = np.stack(test_fps)
  
  origin_train_features = train_df[["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea"]].values
  origin_test_features = test_df[["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea"]].values
  
  train_features = np.append(origin_train_features, train_fps, axis=1)
  test_features = np.append(origin_test_features, test_fps, axis=1)
  target = train_df["MLM"].values
  
  return train_features, target, test_features
  

In [8]:
train_x, train_y, test_x = feature_engineer()

In [9]:
def create_model(param):
  return xgb.XGBRegressor(
    random_state=RANDOM_SEED,
    tree_method="gpu_hist",
    eval_metric="rmse",
    early_stopping_rounds=50,
    **param
  )

In [10]:
def train_model(param, X, y, X_test, trial=None):
    skf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

    val_scores = []
    y_tests = []
    models = []

    for idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):

        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]

        model = create_model(param)
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=1000)

        y_hat_val = model.predict(X_val)
        score = mean_squared_error(y_val, y_hat_val, squared=True)
        val_scores.append(score)
        print(f'Fold: {idx+1}/5 score = {score:.5f}')

        y_tests.append(model.predict(X_test))
        models.append(model)

        if trial:
            trial.report(score, idx)

            if trial.should_prune():
                raise optuna.TrialPruned()

    return val_scores, y_tests, models

In [11]:
def objective_xgb(trial):
    param = {
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
        "max_depth": trial.suggest_int("max_depth", 3, 9),
        "min_child_weight": trial.suggest_int("min_child_weight", 2, 10),
        "eta": trial.suggest_float("eta", 1e-8, 1.0, log=True),
        "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
        "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    }

    if param["booster"] == "dart":
        param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
        param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
        param['rate_drop'] = trial.suggest_float('rate_drop', 1e-8, 1.0, log=True)
        param['skip_drop'] = trial.suggest_float('skip_drop', 1e-8, 1.0, log=True)

    val_scores, y_tests, models = train_model(param, train_x, train_y, test_x, trial)

    return sum(val_scores) / len(val_scores)

In [12]:
study = optuna.create_study(
    sampler=TPESampler(seed=RANDOM_SEED),
    direction='minimize',
    study_name='xgb_tuning',
    pruner=HyperbandPruner(
        min_resource=1, max_resource=8, reduction_factor=3
    ),
)

study.optimize(objective_xgb, n_trials=100)

[I 2023-08-17 16:32:15,863] A new study created in memory with name: xgb_tuning


[0]	validation_0-rmse:50.98355	validation_1-rmse:52.01123
[99]	validation_0-rmse:40.43717	validation_1-rmse:42.38212
Fold: 1/5 score = 1796.24405
[0]	validation_0-rmse:50.48315	validation_1-rmse:53.98357
[99]	validation_0-rmse:40.19955	validation_1-rmse:43.92678
Fold: 2/5 score = 1929.56236
[0]	validation_0-rmse:51.45789	validation_1-rmse:50.11414
[99]	validation_0-rmse:40.98397	validation_1-rmse:40.02995
Fold: 3/5 score = 1602.39658
[0]	validation_0-rmse:51.30739	validation_1-rmse:50.73809
[99]	validation_0-rmse:40.84358	validation_1-rmse:40.63700
Fold: 4/5 score = 1651.36592
[0]	validation_0-rmse:51.71924	validation_1-rmse:48.98052
[99]	validation_0-rmse:40.98622	validation_1-rmse:39.29252
Fold: 5/5 score = 1543.90217


[I 2023-08-17 16:32:55,136] Trial 0 finished with value: 1704.694215564939 and parameters: {'booster': 'dart', 'lambda': 0.007177141927992002, 'alpha': 0.0006155564318973012, 'subsample': 0.24041677639819287, 'colsample_bytree': 0.2403950683025824, 'n_estimators': 100, 'max_depth': 9, 'min_child_weight': 7, 'eta': 0.004619347374377372, 'gamma': 1.4610865886287176e-08, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.00015777981883364995, 'skip_drop': 2.85469785779718e-05}. Best is trial 0 with value: 1704.694215564939.


[0]	validation_0-rmse:51.10227	validation_1-rmse:52.12406
[799]	validation_0-rmse:40.76321	validation_1-rmse:42.51230
Fold: 1/5 score = 1807.29536
[0]	validation_0-rmse:50.58859	validation_1-rmse:54.09132
[799]	validation_0-rmse:40.52118	validation_1-rmse:44.11498
Fold: 2/5 score = 1946.13127
[0]	validation_0-rmse:51.57394	validation_1-rmse:50.22936
[799]	validation_0-rmse:41.27051	validation_1-rmse:40.05899
Fold: 3/5 score = 1604.72286
[0]	validation_0-rmse:51.42142	validation_1-rmse:50.85304
[799]	validation_0-rmse:41.21002	validation_1-rmse:40.67425
Fold: 4/5 score = 1654.39460
[0]	validation_0-rmse:51.84500	validation_1-rmse:49.09808
[799]	validation_0-rmse:41.39511	validation_1-rmse:39.41916
Fold: 5/5 score = 1553.86991


[I 2023-08-17 17:08:55,442] Trial 1 finished with value: 1713.2827987879914 and parameters: {'booster': 'dart', 'lambda': 1.3060231803531604e-07, 'alpha': 2.1734877073417355e-06, 'subsample': 0.4297256589643226, 'colsample_bytree': 0.5104629857953323, 'n_estimators': 800, 'max_depth': 4, 'min_child_weight': 6, 'eta': 0.0005486767416600901, 'gamma': 2.3528990899815284e-08, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 2.734702913886802e-06, 'skip_drop': 6.044730070370796e-08}. Best is trial 0 with value: 1704.694215564939.


...

[I 2023-08-18 02:13:28,616] Trial 97 finished with value: 952.8540828751684 and parameters: {'booster': 'dart', 'lambda': 0.00023381270462447313, 'alpha': 0.0012680628258265715, 'subsample': 0.8636447666282404, 'colsample_bytree': 0.487310741083678, 'n_estimators': 700, 'max_depth': 9, 'min_child_weight': 5, 'eta': 0.019104008674728407, 'gamma': 6.637946881342364e-05, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 1.1940317466932248e-06, 'skip_drop': 0.00011318418946036839}. Best is trial 16 with value: 947.4410144296698.


[0]	validation_0-rmse:50.85758	validation_1-rmse:51.91014
[599]	validation_0-rmse:21.16502	validation_1-rmse:32.25451
Fold: 1/5 score = 1040.14388
[0]	validation_0-rmse:50.34962	validation_1-rmse:53.87061
[599]	validation_0-rmse:21.21266	validation_1-rmse:32.13416
Fold: 2/5 score = 1032.54946
[0]	validation_0-rmse:51.32120	validation_1-rmse:49.99932
[599]	validation_0-rmse:21.57123	validation_1-rmse:29.90469
Fold: 3/5 score = 894.29035
[0]	validation_0-rmse:51.17282	validation_1-rmse:50.60489
[599]	validation_0-rmse:21.72069	validation_1-rmse:29.70532
Fold: 4/5 score = 882.26066


[I 2023-08-18 02:30:50,097] Trial 98 pruned. 


[0]	validation_0-rmse:48.24958	validation_1-rmse:49.60832
[91]	validation_0-rmse:16.38360	validation_1-rmse:33.10494
Fold: 1/5 score = 1081.26265
[0]	validation_0-rmse:47.81361	validation_1-rmse:51.54670
[100]	validation_0-rmse:15.78423	validation_1-rmse:33.11345
Fold: 2/5 score = 1079.25447
[0]	validation_0-rmse:48.66142	validation_1-rmse:47.63958
[96]	validation_0-rmse:16.27432	validation_1-rmse:30.75936
Fold: 3/5 score = 928.72679
[0]	validation_0-rmse:48.53980	validation_1-rmse:48.10601
[102]	validation_0-rmse:15.97012	validation_1-rmse:29.65480
Fold: 4/5 score = 871.62538
[0]	validation_0-rmse:48.95550	validation_1-rmse:46.58581
[137]	validation_0-rmse:13.66005	validation_1-rmse:30.55939
Fold: 5/5 score = 928.71056

[I 2023-08-18 02:31:35,954] Trial 99 finished with value: 977.9159713376946 and parameters: {'booster': 'dart', 'lambda': 5.692784640484324e-05, 'alpha': 0.0033173846724742816, 'subsample': 0.7437612134805827, 'colsample_bytree': 0.459525537349439, 'n_estimators': 700, 'max_depth': 9, 'min_child_weight': 5, 'eta': 0.08340681537461535, 'gamma': 0.00012901094624464385, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 9.282859980166626e-07, 'skip_drop': 0.00017141964300690493}. Best is trial 16 with value: 947.4410144296698.


In [13]:
trial = study.best_trial

In [14]:
print(trial.params)

{'booster': 'dart', 'lambda': 0.0026377247595962727, 'alpha': 7.483780511102761e-05, 'subsample': 0.8280537243135858, 'colsample_bytree': 0.37588574594589486, 'n_estimators': 400, 'max_depth': 7, 'min_child_weight': 8, 'eta': 0.05393761227911353, 'gamma': 4.291867724531988e-05, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 5.145690154427874e-07, 'skip_drop': 1.3546656519467045e-06}


In [16]:
best_param = {
  'booster': 'dart',
  'lambda': 0.0026377247595962727,
  'alpha': 7.483780511102761e-05,
  'subsample': 0.8280537243135858,
  'colsample_bytree': 0.37588574594589486,
  'n_estimators': 400,
  'max_depth': 7,
  'min_child_weight': 8,
  'eta': 0.05393761227911353,
  'gamma': 4.291867724531988e-05,
  'grow_policy': 'depthwise',
  'sample_type': 'weighted',
  'normalize_type': 'tree',
  'rate_drop': 5.145690154427874e-07,
  'skip_drop': 1.3546656519467045e-06}

val_scores, y_tests, models = train_model(best_param, train_x, train_y, test_x)

[0]	validation_0-rmse:49.32643	validation_1-rmse:50.54414
[167]	validation_0-rmse:19.66857	validation_1-rmse:32.25208
Fold: 1/5 score = 1037.66947
[0]	validation_0-rmse:48.86619	validation_1-rmse:52.45612
[183]	validation_0-rmse:19.16252	validation_1-rmse:32.42064
Fold: 2/5 score = 1045.10372
[0]	validation_0-rmse:49.76686	validation_1-rmse:48.50828
[222]	validation_0-rmse:18.64541	validation_1-rmse:30.01686
Fold: 3/5 score = 894.87590
[0]	validation_0-rmse:49.61942	validation_1-rmse:49.15592
[241]	validation_0-rmse:18.04355	validation_1-rmse:29.67055
Fold: 4/5 score = 876.60151
[0]	validation_0-rmse:50.05397	validation_1-rmse:47.47320
[248]	validation_0-rmse:18.02924	validation_1-rmse:29.77147
Fold: 5/5 score = 882.95447


In [23]:
importance = models[-1].get_booster().get_score(importance_type="weight")
tuples = [(k, importance[k]) for k in importance]
tuples.sort(key=lambda x: x[1])

label, values = zip(*tuples)

plt.rcParams["font.family"] = "MalGun Gothic"
plt.figure(figsize=(10, 12))
plt.barh(label[:100], values[:100])
plt.show()

In [24]:
submit = pd.read_csv("./data/sample_submission.csv")
submit["MLM"] = y_tests[-1]
submit.to_csv("xgboost_optuna.csv", index=False)